In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from sklearn.metrics import mean_squared_error
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if len(data.shape) == 1 else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("var%d(t-%d)" % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [("var%d(t)" % (j + 1)) for j in range(n_vars)]
        else:
            names += [("var%d(t+%d)" % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [3]:
df = pd.read_csv(
    "../assets/datasets/time_series_solar.csv",
    parse_dates=["Datetime"],
    index_col="Datetime",
)


In [4]:
# Resample the data to daily frequency
df = df.resample("D").sum()

values = df["Incoming Solar"].values

data = series_to_supervised(values, 3)
print(data)

      var1(t-3)  var1(t-2)  var1(t-1)  var1(t)
3        1381.5     3953.2     3098.1   2213.9
4        3953.2     3098.1     2213.9   1338.8
5        3098.1     2213.9     1338.8   3671.5
6        2213.9     1338.8     3671.5   4193.7
7        1338.8     3671.5     4193.7   4213.8
...         ...        ...        ...      ...
2187     4113.6     2134.2     1250.2   1034.2
2188     2134.2     1250.2     1034.2   2182.3
2189     1250.2     1034.2     2182.3   3384.5
2190     1034.2     2182.3     3384.5    478.2
2191     2182.3     3384.5      478.2   2554.8

[2189 rows x 4 columns]


In [5]:
scaler = MinMaxScaler(feature_range=(-1, 1))
train, test = train_test_split(data, test_size=0.2, shuffle=False)
train = scaler.fit_transform(train)
test = scaler.transform(test)

X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

X_train = torch.from_numpy(X_train).type(torch.Tensor)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor).view(-1)
y_test = torch.from_numpy(y_test).type(torch.Tensor).view(-1)

In [6]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.activation = nn.Tanh()

    def forward(self, x):
        out = self.activation(self.fc1(x))
        out = self.fc2(out)
        return out

In [7]:
input_dim = X_train.shape[1]
hidden_dim = 32
model = FeedForwardNN(input_dim, hidden_dim)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(-1)
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.8639694452285767
Epoch: 10, Loss: 0.11101436614990234
Epoch: 20, Loss: 0.13403956592082977
Epoch: 30, Loss: 0.11255411058664322
Epoch: 40, Loss: 0.10725408792495728
Epoch: 50, Loss: 0.1079149842262268
Epoch: 60, Loss: 0.10655283182859421
Epoch: 70, Loss: 0.10635124146938324
Epoch: 80, Loss: 0.10632209479808807
Epoch: 90, Loss: 0.10620933026075363
Epoch: 100, Loss: 0.10612325370311737
Epoch: 110, Loss: 0.10605306178331375
Epoch: 120, Loss: 0.10598137229681015
Epoch: 130, Loss: 0.10590774565935135
Epoch: 140, Loss: 0.10583289712667465
Epoch: 150, Loss: 0.10575704276561737
Epoch: 160, Loss: 0.10568017512559891
Epoch: 170, Loss: 0.10560228675603867
Epoch: 180, Loss: 0.10552342981100082
Epoch: 190, Loss: 0.10544358938932419


In [8]:
model.eval()
y_pred = model(X_test).reshape(-1)
test_loss = loss_fn(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.09483013302087784


In [9]:
y_pred = model(X_test).detach().numpy()
y_true = y_test.detach().numpy()
y_train = y_train.detach().numpy()

rmse_sklearn = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"RMSE: {rmse_sklearn}")

mape_sktime = MeanAbsolutePercentageError(symmetric=False)
mape = mape_sktime(y_true, y_pred)
print(f"MAPE: {mape}")

smape_sktime = MeanAbsolutePercentageError(symmetric=True)
smape = smape_sktime(y_true, y_pred)
print(f"SMAPE: {smape}")

mase_sktime = MeanAbsoluteScaledError()
mase = mase_sktime(y_true, y_pred, y_train=y_train)
print(f"MASE: {mase}")

RMSE: 0.30794501622997217
MAPE: 1.8008296441521916
SMAPE: 0.6581115928657892
MASE: 0.9074650616451796
